In [ ]:
pip install pyautogen

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.2/345.2 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.6/375.6 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.9 MB/s eta 0:00:00


In [ ]:
pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 3.9 MB/s eta 0:00:00


In [ ]:
from groq import Groq

In [ ]:
import os

os.environ["GROQ_API_KEY"] = "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"

In [ ]:
import datetime

today = datetime.datetime.now().date()

In [ ]:
task = f"Today is {today}"\
"Do analysis of the stock price performance of Nvidia in the past month and suggest some threshold values for price changes"\
"or technical indicators (e.g., moving average, RSI) so that I can set up alerts for significant movements. This will involve analyzing trends,"\
"identifying key support and resistance levels, and determining an appropriate threshold for alerting based on historical data patterns."

## Build a group chat

This group chat will include these agents:

1. **User_proxy** or **Admin**: to allow the user to comment on the report and ask the writer to refine it.
2. **Planner**: to determine relevant information needed to complete the task.
3. **Engineer**: to write code using the defined plan by the planner.
4. **Executor**: to execute the code written by the engineer.
5. **Writer**: to write the report.

In [ ]:
from autogen import ConversableAgent,AssistantAgent,GroupChatManager,GroupChat

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:
from autogen.coding import LocalCommandLineCodeExecutor

In [ ]:
def get_stock_prices(stock_symbols, start_date, end_date):
    """Get the stock prices for the given stock symbols between
    the start and end dates.

    Args:
        stock_symbols (str or list): The stock symbols to get the
        prices for.
        start_date (str): The start date in the format
        'YYYY-MM-DD'.
        end_date (str): The end date in the format 'YYYY-MM-DD'.

    Returns:
        pandas.DataFrame: The stock prices for the given stock
        symbols indexed by date, with one column per stock
        symbol.
    """
    import yfinance

    stock_data = yfinance.download(
        stock_symbols, start=start_date, end=end_date
    )
    return stock_data.get("Close")

In [ ]:
def plot_stock_prices(stock_prices, filename):
    """Plot the stock prices for the given stock symbols.

    Args:
        stock_prices (pandas.DataFrame): The stock prices for the
        given stock symbols.
    """
    import matplotlib.pyplot as plt

    plt.figure(figsize=(10, 5))
    for column in stock_prices.columns:
        plt.plot(
            stock_prices.index, stock_prices[column], label=column
                )
    plt.title("Stock Prices")
    plt.xlabel("Date")
    plt.ylabel("Price")
    plt.grid(True)
    plt.savefig(filename)

In [ ]:
executor1 = LocalCommandLineCodeExecutor(
    timeout=60,
    work_dir="coding",
    functions=[get_stock_prices, plot_stock_prices],
)

In [ ]:
user_proxy = ConversableAgent(
    name="Admin",
    system_message="Give the task, and send "
    "instructions to writer to refine Report.",
    code_execution_config=False,
    llm_config={"config_list": [{"model": "llama3-8b-8192","api_type":"groq", "api_key": os.environ["GROQ_API_KEY"]}]},
    human_input_mode="ALWAYS",
)

In [ ]:
planner = ConversableAgent(
    name="Planner",
    system_message="Given a task, please determine "
    "what information is needed to complete the task. "
    "Please note that the information will all be retrieved using"
    " Python code. Please only suggest information that can be "
    "retrieved using Python code. "
    "After each step is done by others, check the progress and "
    "instruct the remaining steps. If a step fails, try to "
    "workaround",
    description="Planner. Given a task, determine what "
    "information is needed to complete the task. "
    "After each step is done by others, check the progress and "
    "instruct the remaining steps,do not provide the code "
     "Return 'TERMINATE' when the task is done.",
    llm_config={"config_list": [{"model": "llama3-8b-8192","api_type":"groq", "api_key": os.environ["GROQ_API_KEY"]}]},
)

In [ ]:
engineer = AssistantAgent(
    name="Engineer",
    llm_config={"config_list": [{"model": "llama3-8b-8192","api_type":"groq", "api_key": os.environ["GROQ_API_KEY"]}]},
    description="An engineer that writes accurate code based on the plan "
    "provided by the planner.",
)

In [ ]:
executor = ConversableAgent(
    name="Executor",
    system_message="Execute and debug the code written by the "
    "engineer in python and report the result.",
    human_input_mode="NEVER",
    code_execution_config={"executor": executor1},
)

In [ ]:
writer = ConversableAgent(
    name="Writer",
    llm_config={"config_list": [{"model": "llama3-8b-8192","api_type":"groq", "api_key": os.environ["GROQ_API_KEY"]}]},
    system_message="Writer."
    "Please write report in markdown format (with relevant titles)"
    " and put the content in pseudo ```md``` code block. "
    "You take feedback from the admin and refine your report.",
    description="Writer."
    "Write report based on the code execution results from engineer and take "
    "feedback from the admin to refine the report."
    "Give a conclusion of the task just before terminating"
    "Return TERMINATE when the task is done"
)

In [ ]:
# Updated GroupChat configuration based on the defined roles and transitions.
groupchat = GroupChat(
    agents=[user_proxy, planner, engineer, executor, writer],
    messages=[],
    max_round=20,
    allowed_or_disallowed_speaker_transitions={
        user_proxy: [planner],

        planner: [engineer],

        engineer: [executor],

        executor: [writer],

        writer: [user_proxy, planner]
    },
    speaker_transitions_type="allowed",  # Defining allowed transitions explicitly.
)



In [ ]:
manager = GroupChatManager(
    groupchat=groupchat, llm_config={"config_list": [{"model": "llama3-8b-8192","api_type":"groq", "api_key": os.environ["GROQ_API_KEY"]}]}
)


In [ ]:
groupchat_result = user_proxy.initiate_chat(
    manager,
    message=task,
)

Admin (to chat_manager):

Today is 2024-09-21Do analysis of the stock price performance of Nvidia in the past month and suggest some threshold values for price changesor technical indicators (e.g., moving average, RSI) so that I can set up alerts for significant movements. This will involve analyzing trends,identifying key support and resistance levels, and determining an appropriate threshold for alerting based on historical data patterns.

--------------------------------------------------------------------------------

Next speaker: Planner


>>>>>>>> USING AUTO REPLY...
Planner (to chat_manager):

To analyze the stock price performance of Nvidia in the past month and suggest some threshold values for price changes or technical indicators, we will need to retrieve the stock data using Python with a library such as yfinance. This will allow us to get historical data on the stock's price movement over a specified period.

Here is a general approach:

Step 1: Retrieve the historical da

In [ ]:
from pprint import pprint

pprint(groupchat_result)


ChatResult(chat_id=None,
           chat_history=[{'content': 'Today is 2024-09-21Do analysis of the '
                                     'stock price performance of Nvidia in the '
                                     'past month and suggest some threshold '
                                     'values for price changesor technical '
                                     'indicators (e.g., moving average, RSI) '
                                     'so that I can set up alerts for '
                                     'significant movements. This will involve '
                                     'analyzing trends,identifying key support '
                                     'and resistance levels, and determining '
                                     'an appropriate threshold for alerting '
                                     'based on historical data patterns.',
                          'name': 'Admin',
                          'role': 'assistant'},
                         